### Import potrzebnych bibliotek

In [ ]:
import pandas as pd
import numpy as np
import random as rd
import matplotlib.pyplot as plt
import seaborn as sns
import time
from tqdm import tqdm

### Funkcje obliczające długość 

In [ ]:
#funkcja obliczająca odległość między dwoma wierzchołkami z matematycznym zaokrągleniem
def distance(x1,x2,y1,y2):
    return np.floor(np.sqrt((x1-x2)**2+(y1-y2)**2)+0.5)

In [ ]:
#funkcja obliczająca całkowitą długość cyklu
def distance_sum(cycle,matrix):
    distance_list = [matrix[cycle[i]][cycle[0]] if i==len(cycle)-1 else matrix[cycle[i]][cycle[i+1]]
                    for i in range(len(cycle))]
    
    return sum(distance_list)

### Wczytanie obu instancji problemu

In [ ]:
#wczytanie pierwszej instancji (kroA200)

instance = pd.read_csv('kroA200.tsp', sep=' ', names=['node', 'x', 'y'], skiprows=6, skipfooter=1,
                      engine='python')
kroA200 = []

for index1, row1 in instance.iterrows():
    tmp = []
    for index2, row2 in instance.iterrows():
        if (index1==index2):
            tmp.append(0)
        else:
            tmp.append(distance(row1['x'],row2['x'],row1['y'],row2['y']))
    kroA200.append(tmp)
    
np.array(kroA200)

In [ ]:
#wczytanie drugiej instancji (kroB200)

instance2 = pd.read_csv('kroB200.tsp', sep=' ', names=['node', 'x', 'y'], skiprows=6, skipfooter=1,
                      engine='python')

kroB200 = []

for index1, row1 in instance2.iterrows():
    tmp = []
    for index2, row2 in instance2.iterrows():
        if (index1==index2):
            tmp.append(0)
        else:
            tmp.append(distance(row1['x'],row2['x'],row1['y'],row2['y']))
    kroB200.append(tmp)

np.array(kroB200)

Losowanie wierzchołków startowych

In [ ]:
#losowanie pierwszego wierzchołka startowego - drugi wyznaczany jest jako najdalszy od pierwszego
def get_start_nodes(matrix):
    first_start=rd.randint(0, len(matrix)-1)
    second_start=np.argmax(matrix[first_start])
    return first_start,second_start

Utworzenie losowych cykli

In [ ]:
def create_random_cycles(instance):
    vertices = np.arange(0, len(instance))
    rd.shuffle(vertices)
    
    split_index = len(instance) // 2
    
    # return cycle1 and cycle2
    return vertices[:split_index], vertices[split_index:]

Obliczenie delty funkcji celu

In [ ]:
def calculate_delta(cycle1, cycle2, matrix, move):
    # wymiana wierzchołków między cyklami
    if move[0] == 0:
        index1, index2 = move[1][0], move[1][1]
        
        node1, node2 = cycle1[index1], cycle2[index2]
        node1_before, node2_before = cycle1[index1-1], cycle2[index2-1]
        node1_after, node2_after = cycle1[(index1+1) % len(cycle1)], cycle2[(index2+1) % len(cycle2)]
        
        delta = matrix[node1_before][node2] + matrix[node2][node1_after] + \
                matrix[node2_before][node1] + matrix[node1][node2_after] - \
                matrix[node1_before][node1] - matrix[node1][node1_after] - \
                matrix[node2_before][node2] - matrix[node2][node2_after]
     
    # wymiana wierzchołków w cyklu
    elif move[0] == 1:
        cycle = cycle1 if not move[1][0] else cycle2
        
        index1, index2 = move[1][1], move[1][2]
        
        node1, node2 = cycle[index1], cycle[index2]
        node1_before, node2_before = cycle[index1-1], cycle[index2-1]
        node1_after, node2_after = cycle[(index1+1) % len(cycle)], cycle[(index2+1) % len(cycle)]
        
        old_distance = matrix[node1][node1_before] + matrix[node1][node1_after] + matrix[node2][node2_before] + matrix[node2][node2_after]
        new_distance = matrix[node1][node2_before] + matrix[node1][node2_after] + matrix[node2][node1_before] + matrix[node2][node1_after]
        
        delta = new_distance - old_distance
        
    # wymiana krawędzi w cyklu
    else:
        cycle = cycle1 if not move[1][0] else cycle2
        index1, index1_2, index2, index2_2 = move[1][1:]
        node1, node1_2, node2, node2_2 = cycle[index1], cycle[index1_2], cycle[index2], cycle[index2_2]
        
        delta = matrix[node1][node2] + matrix[node1_2][node2_2] - matrix[node1][node1_2] - matrix[node2][node2_2]
        
    return delta

Tworzenie cykli metodą zachłanną

In [ ]:
# matrix to np.array(kroA200) lub np.array(kroB200)
def greedy_cycle(matrix, first_start, second_start):
    
    # utwórz dwa cykle i listę dostępnych wierzchołków
    cycle1=[first_start]
    cycle2=[second_start]
    
    # True tam, gdzie jest dostępny wierzchołek, False tam, gdzie jest już wstawiony w cyklu
    remaining = np.ones((matrix.shape[0],), dtype=bool)
    remaining[first_start] = False
    remaining[second_start] = False
    
    # oblicz długość obu cykli
    cycle1_length = matrix.shape[0] // 2
    cycle2_length = matrix.shape[0] - cycle1_length
    
    # dla każdego cyklu stwórz cykl metodą zachłanną
    for cycle, length in [[cycle1, cycle1_length], [cycle2, cycle2_length]]:
        
        # dodaj najbliższego sąsiada dla wierzchołka początkowego
        min_distance = np.min(matrix[cycle[0]][remaining])
        # znajdź wierzchołki, dla których odległość jest najmniejsza
        min_nodes = np.squeeze(np.argwhere(matrix[cycle[0]] == min_distance), axis=-1)
        # wybierz pierwszy wierzchołek
        chosen_node = min_nodes[0]
        # dodaj wierzchołek do cyklu
        cycle.append(chosen_node)
        # zmień wartość w liście remaining na pozycji chosen_node na False
        remaining[chosen_node] = False
        
        # dodawanie pozostałych wierzchołków do cyklu
        for i in range(length-2):
            # wybierz pierwszy wierzchołek
            best_start = cycle[0]
            # wybierz pierwszy możliwy wierzchołek końcowy dla wybranego
            best_end = np.argwhere(remaining)[0, 0]
            # delta wstawienia między wierzchołek 0 a 1
            best_delta = matrix[best_start, best_end] + matrix[best_end, cycle[1]] - \
                         matrix[best_start, cycle[1]]
            
            # dla wszystkich wierzchołków w cyklu
            for j in range(len(cycle)):
                # dla wszystkich możliwych wierzchołków
                for k in range(matrix.shape[0]):
                    if remaining[k]:
                        delta = matrix[cycle[j], k] + matrix[k, cycle[(j+1) % len(cycle)]] - \
                        matrix[cycle[j], cycle[(j+1) % len(cycle)]]
                        
                        # aktualizacja najlepszego wierzchołka do wstawienia
                        if delta < best_delta:
                            best_start = cycle[j]
                            best_end = k
                            best_delta = delta
            
            # wstaw najlepszy znaleziony wierzchołek na pozycji za najlepszym startowym
            cycle.insert(cycle.index(best_start) + 1, best_end)
            remaining[best_end] = False
            
    return cycle1, cycle2

In [ ]:
def calculate_regret_with_weight(list_nodes):
    list_nodes = np.array(list_nodes)
    difference = list_nodes[:,2]-1.2*(list_nodes[:,1])
    best_result_index = np.argmax(difference)
    return int(list_nodes[best_result_index][0])

In [ ]:
def nodes_with_best_given_index(matrix,cycle,remaining,given_index):
    result_nodes = []
    for node_to_add in remaining:
        best1 = 9999999999998
        best2 = 9999999999999
        index = -1
        for i in range (len(cycle)):
            if i!=len(cycle)-1:
                tmp_distance = matrix[cycle[i]][node_to_add]+ matrix[cycle[i+1]][node_to_add] - matrix[cycle[i]][cycle[i+1]]
            else:
                tmp_distance = matrix[cycle[i]][node_to_add]+ matrix[cycle[0]][node_to_add] - matrix[cycle[i]][cycle[0]]
            if tmp_distance < best2:
                if tmp_distance < best1:
                    best2 = best1
                    best1 = tmp_distance
                    index = i
                else:
                    best2 = tmp_distance
        if index == given_index:
            result_nodes.append([node_to_add,best1,best2])
    return result_nodes

In [ ]:
def start_greedy(matrix, cycle, remaining):
    min_distance = 999999999
    index = -1
    min_node = cycle[0]
    for node_to_add in remaining:
        for i in range (len(cycle)):
            if i!=len(cycle)-1:
                tmp_distance = matrix[cycle[i]][node_to_add]+ matrix[cycle[i+1]][node_to_add] - matrix[cycle[i]][cycle[i+1]]
            else:
                tmp_distance = matrix[cycle[i]][node_to_add]+ matrix[cycle[0]][node_to_add] - matrix[cycle[i]][cycle[0]]
            if tmp_distance < min_distance:
                min_distance = tmp_distance
                min_node = node_to_add
                index = i
    return min_node, index

In [ ]:
def greedy_regret_with_weight(matrix, cycle, remaining):
    best_node_to_add, best_index = start_greedy(matrix, cycle, remaining)
    nodes_on_the_same_position = nodes_with_best_given_index(matrix, cycle, remaining, best_index)
    return calculate_regret_with_weight(nodes_on_the_same_position), best_index

In [ ]:
def calculate_greedy_regret_with_weight(matrix, cycle1, cycle2, remaining):

    while (remaining):
        node_to_add, index = greedy_regret_with_weight(matrix, cycle1, remaining)
        index = int(index)
        cycle1.insert(index+1, node_to_add)
        remaining.remove(node_to_add)
        if not remaining:
             break
        node_to_add, index = greedy_regret_with_weight(matrix, cycle2, remaining)
        index = int(index)
        cycle2.insert(index+1, node_to_add)
        remaining.remove(node_to_add)
        
    return cycle1, cycle2

Generowanie ruchów

In [ ]:
# change_type -> 1 dla wymiany wierzchołków w cyklu, 2 dla wymiany krawędzi w cyklu
def generate_moves(cycle1, cycle2, change_type):
    
    # ruchy międzytrasowe
    moves1 = [(0, (index1, index2)) for index1 in range(len(cycle1)) for index2 in range(len(cycle2))]
    
    # ruchy wewnątrztrasowe
    moves2 = []
    # wymiana wierzchołków w cyklu
    if change_type == 1:
        moves2 = [(1, (cycle_number, index1, index2)) for cycle_number, cycle in enumerate([cycle1, cycle2]) 
                  for index1 in range(len(cycle)) for index2 in range(len(cycle)) if index1 != index2]
    
    moves3 = []
    # wymiana krawędzi w cyklu
    if change_type == 2:
        for cycle_number, cycle in enumerate([cycle1, cycle2]):
            for index1 in range(len(cycle)):
                for index2 in range(len(cycle)):
                    if abs(index1 - index2) < 2:
                        continue
                        
                    if index1 == len(cycle) - 1:
                        index1_end = 0
                    else:
                        index1_end = index1 + 1
                        
                    if index2 == len(cycle) - 1:
                        index2_end = 0
                    else:
                        index2_end = index2 + 1
                        
                    move = (2, (cycle_number, index1, index1_end, index2, index2_end))
                    moves3.append(move)
                    
    return moves1 + moves2 + moves3

Wykonaj ruch

In [ ]:
def apply_move(cycle1, cycle2, move):
    # ruch pomiędzy cyklami
    if move[0] == 0:
        temp = cycle1[move[1][0]]
        cycle1[move[1][0]] = cycle2[move[1][1]]
        cycle2[move[1][1]] = temp
        
    # ruch w cyklu -> wymiana wierzchołków
    elif move[0] == 1:
        cycle = cycle2 if move[1][0] else cycle1

        temp = cycle[move[1][1]]
        cycle[move[1][1]] = cycle[move[1][2]]
        cycle[move[1][2]] = temp
     
    # ruch w cyklu -> wymiana krawędzi
    elif move[0] == 2:
        cycle = cycle2 if move[1][0] else cycle1
        index1, index1_2, index2, index2_2 = move[1][1:]
        if index1_2 == 0:
            c1 = [cycle[index1_2]]
            c2 = cycle[index2_2:index1 + 1]
            c3 = cycle[index2:index1_2:-1]
        elif index2_2 == 0:
            c1 = cycle[:index1_2]
            c2 = cycle[index2:index1:-1]
            c3 = []
        else:
            c1 = cycle[:min(index1_2, index2_2)]
            c2 = cycle[max(index1_2, index2_2)-1 : min(index1_2, index2_2)-1 : -1]
            c3 = cycle[max(index1_2, index2_2):]
        
        if type(c1) is np.ndarray:
            c1 = c1.tolist()
        if type(c2) is np.ndarray:
            c2 = c2.tolist()
        if type(c3) is np.ndarray:
            c3 = c3.tolist()
            
        if not isinstance(c1, list):
            c1 = [c1]
        if not isinstance(c2, list):
            c2 = [c2]
        if not isinstance(c3, list):
            c3 = [c3]
        
        cycle = c1 + c2 + c3
        
        if not move[1][0]:
            cycle1 = cycle
        else:
            cycle2 = cycle
            
    return cycle1, cycle2

Zachłanne przeszukiwanie lokalne

In [ ]:
# change_type -> 1 dla wymiany wierzchołków w cyklu, 2 dla wymiany krawędzi w cyklu
def greedy_local_search(cycle1, cycle2, matrix, change_type):
    start_time = time.time()
    
    cycle1_copy = cycle1.copy()
    cycle2_copy = cycle2.copy()
    
    moves = generate_moves(cycle1_copy, cycle2_copy, change_type)
    
    while True:
        no_better_moves = True
        random_bias = rd.randrange(0, len(moves))
        random_direction = rd.choice([-1, 1])
        
        for i in range(len(moves)):
            move = moves[random_direction * ((i + random_bias) % len(moves))]
            delta = calculate_delta(cycle1_copy, cycle2_copy, matrix, move)
            if delta < 0:
                cycle1_copy, cycle2_copy = apply_move(cycle1_copy, cycle2_copy, move)
                no_better_moves = False
                break

        if no_better_moves:
            break

    total_time_seconds = time.time() - start_time
    return cycle1_copy, cycle2_copy, total_time_seconds

In [ ]:
# start_solution -> 'random' lub 'not_random'
def multiple_start_local_search(matrix, iterations=100, start_solution='random'):
    start_time = time.time()
    
    cycle1_best, cycle2_best = None, None
    best_result = 999999999999999
    
    for i in range(iterations):
        if start_solution == 'random':
            cycle1, cycle2 = create_random_cycles(matrix)
        else:
            first_start, second_start = get_start_nodes(matrix)
            cycle1, cycle2 = greedy_cycle(np.array(matrix), first_start, second_start)
        
        cycle1_res, cycle2_res, _ = greedy_local_search(cycle1, cycle2, matrix, 2)
        
        result = distance_sum(cycle1_res, matrix) + distance_sum(cycle2_res, matrix)
        if result < best_result:
            cycle1_best, cycle2_best = cycle1_res, cycle2_res
            best_result = result
        
    total_time_seconds = time.time() - start_time
    
    return cycle1_best, cycle2_best, total_time_seconds

In [ ]:
c1_b, c2_b, tts = multiple_start_local_search(kroB200, 100, 'random')

In [ ]:
c1_b

In [ ]:
c2_b

In [ ]:
tts

Perturbacje ILS1

In [ ]:
def perturbation1(cycle1, cycle2, moves_number = 5):
    for m in range(moves_number):
        moves = generate_moves(cycle1, cycle2, 2)
        move = rd.sample(moves, 1)
        cycle1, cycle2 = apply_move(cycle1, cycle2, move)
        
    return cycle1, cycle2

ILS1

In [ ]:
# start_solution -> 'random' lub 'not_random'
def ils1(matrix, duration_time, start_solution='random'):
    start_time = time.time()    

    if start_solution == 'random':
        cycle1, cycle2 = create_random_cycles(matrix)
    else:
        first_start, second_start = get_start_nodes(matrix)
        cycle1, cycle2 = greedy_cycle(np.array(matrix), first_start, second_start)

    cycle1_res, cycle2_res, _ = greedy_local_search(cycle1, cycle2, matrix, 2)
    
    cycle1_best, cycle2_best = cycle1_res.copy(), cycle2_res.copy()
    best_result = distance_sum(cycle1_res, matrix) + distance_sum(cycle2_res, matrix)
        
    while time.time() - start_time < duration_time:
        cycle1_copy, cycle2_copy = cycle1_best.copy(), cycle2_best.copy()
        cycle1_p, cycle2_p = perturbation1(cycle1_copy, cycle2_copy)
        cycle1_res, cycle2_res, _ = greedy_local_search(cycle1_p, cycle2_p, matrix, 2)
        
        result = distance_sum(cycle1_res, matrix) + distance_sum(cycle2_res, matrix)
        
        if result < best_result:
            cycle1_best, cycle2_best = cycle1_res, cycle2_res
            best_result = result
        
    total_time_seconds = time.time() - start_time
    
    return cycle1_best, cycle2_best, total_time_seconds

In [ ]:
c1_b, c2_b, tts = ils1(kroB200, 10, 'random')

In [ ]:
tts

In [ ]:
distance_sum(c1_b, kroB200) + distance_sum(c2_b, kroB200)

Perturbacja ILS2

In [ ]:
def perturbation2(cycle1, cycle2, delete_percent=20):
    how_many_delete=len(cycle1)*delete_percent/100
    cycle1_to_delete = rd.sample(cycle1, int(how_many_delete))
    cycle2_to_delete = rd.sample(cycle2, int(how_many_delete))
    for node in cycle1_to_delete:
        cycle1.remove(node)
    for node in cycle2_to_delete:
        cycle2.remove(node)
    return cycle1, cycle2, cycle1_to_delete+cycle2_to_delete

In [ ]:
def ils2(matrix, duration_time, start_solution='random'):
    start_time = time.time()    

    if start_solution == 'random':
        cycle1, cycle2 = create_random_cycles(matrix)
        cycle1, cycle2, _ = greedy_local_search(cycle1, cycle2, matrix, 2)
    else:
        first_start, second_start = get_start_nodes(matrix)
        cycle1, cycle2 = greedy_cycle(np.array(matrix), first_start, second_start)

    
    cycle1_best, cycle2_best = cycle1.copy(), cycle2.copy()
    best_result = distance_sum(cycle1_best, matrix) + distance_sum(cycle2_best, matrix)
        
    while time.time() - start_time < duration_time:
        cycle1_copy, cycle2_copy = cycle1_best.copy(), cycle2_best.copy()
        cycle1_p, cycle2_p, remaining = perturbation2(cycle1_copy, cycle2_copy)
        cycle1_res, cycle2_res = calculate_greedy_regret_with_weight(matrix, cycle1_p, cycle2_p, remaining)
        
        result = distance_sum(cycle1_res, matrix) + distance_sum(cycle2_res, matrix)
        
        if result < best_result:
            cycle1_best, cycle2_best = cycle1_res, cycle2_res
            best_result = result
        
    total_time_seconds = time.time() - start_time
    
    return cycle1_best, cycle2_best, total_time_seconds

In [ ]:
c1_b, c2_b, tts = ils2(kroB200, 10, 'random')

In [ ]:
tts

In [ ]:
distance_sum(c1_b, kroB200) + distance_sum(c2_b, kroB200)

In [ ]:
print(len(c2_b))

In [ ]:
def ils2a(matrix, duration_time, start_solution='random'):
    start_time = time.time()    

    if start_solution == 'random':
        cycle1, cycle2 = create_random_cycles(matrix)
        cycle1, cycle2, _ = greedy_local_search(cycle1, cycle2, matrix, 2)
    else:
        first_start, second_start = get_start_nodes(matrix)
        cycle1, cycle2 = greedy_cycle(np.array(matrix), first_start, second_start)

    
    cycle1_best, cycle2_best = cycle1.copy(), cycle2.copy()
    best_result = distance_sum(cycle1_best, matrix) + distance_sum(cycle2_best, matrix)
        
    while time.time() - start_time < duration_time:
        cycle1_copy, cycle2_copy = cycle1_best.copy(), cycle2_best.copy()
        cycle1_p, cycle2_p, remaining = perturbation2(cycle1_copy, cycle2_copy)
        cycle1_res, cycle2_res = calculate_greedy_regret_with_weight(matrix, cycle1_p, cycle2_p, remaining)
        cycle1_res, cycle2_res, _ = greedy_local_search(cycle1_res, cycle2_res, matrix, 2)
        
        result = distance_sum(cycle1_res, matrix) + distance_sum(cycle2_res, matrix)
        
        if result < best_result:
            cycle1_best, cycle2_best = cycle1_res, cycle2_res
            best_result = result
        
    total_time_seconds = time.time() - start_time
    
    return cycle1_best, cycle2_best, total_time_seconds

In [ ]:
c1_b, c2_b, tts = ils2(kroB200, 10, 'random')

In [ ]:
tts

In [ ]:
distance_sum(c1_b, kroB200) + distance_sum(c2_b, kroB200)

In [ ]:
results_kroA200 = [[],[],[],[]]
results_kroB200 = [[],[],[],[]]
results_kroA200_time = [[],[],[],[]]
results_kroB200_time = [[],[],[],[]]

In [ ]:
for i in tqdm(range(10)):
    cycle1_best, cycle2_best, msls_time = multiple_start_local_search(kroA200, 100, 'random')
    result = distance_sum(cycle1_best, kroA200) + distance_sum(cycle2_best, kroA200)
    results_kroA200[0].append([result, cycle1_best, cycle2_best])
    results_kroA200_time[0].append(msls_time)
    
    cycle1_best, cycle2_best, msls_time = multiple_start_local_search(kroB200, 100, 'random')
    result = distance_sum(cycle1_best, kroB200) + distance_sum(cycle2_best, kroB200)
    results_kroB200[0].append([result, cycle1_best, cycle2_best])
    results_kroB200_time[0].append(msls_time)

In [ ]:
for i in tqdm(range(10)):
    cycle1_best, cycle2_best, msls_time = ils1(kroA200, 80, 'random')
    result = distance_sum(cycle1_best, kroA200) + distance_sum(cycle2_best, kroA200)
    results_kroA200[1].append([result, cycle1_best, cycle2_best])
    results_kroA200_time[1].append(msls_time)
    
    cycle1_best, cycle2_best, msls_time = ils1(kroB200, 80, 'random')
    result = distance_sum(cycle1_best, kroB200) + distance_sum(cycle2_best, kroB200)
    results_kroB200[1].append([result, cycle1_best, cycle2_best])
    results_kroB200_time[1].append(msls_time)
    
    cycle1_best, cycle2_best, msls_time = ils2(kroA200, 80, 'random')
    result = distance_sum(cycle1_best, kroA200) + distance_sum(cycle2_best, kroA200)
    results_kroA200[2].append([result, cycle1_best, cycle2_best])
    results_kroA200_time[2].append(msls_time)
    
    cycle1_best, cycle2_best, msls_time = ils2(kroB200, 80, 'random')
    result = distance_sum(cycle1_best, kroB200) + distance_sum(cycle2_best, kroB200)
    results_kroB200[2].append([result, cycle1_best, cycle2_best])
    results_kroB200_time[2].append(msls_time)
    
    cycle1_best, cycle2_best, msls_time = ils2a(kroA200, 80, 'random')
    result = distance_sum(cycle1_best, kroA200) + distance_sum(cycle2_best, kroA200)
    results_kroA200[3].append([result, cycle1_best, cycle2_best])
    results_kroA200_time[3].append(msls_time)
    
    cycle1_best, cycle2_best, msls_time = ils2a(kroB200, 80, 'random')
    result = distance_sum(cycle1_best, kroB200) + distance_sum(cycle2_best, kroB200)
    results_kroB200[3].append([result, cycle1_best, cycle2_best])
    results_kroB200_time[3].append(msls_time)

In [ ]:
print("MSLS kroA200 time:")
print("Mean:", np.mean(results_kroA200_time[0]))
print("Max:", np.max(results_kroA200_time[0]))
print("Min:", np.min(results_kroA200_time[0]))
print("MSLS kroB200 time:")
print("Mean:", np.mean(results_kroB200_time[0]))
print("Max:", np.max(results_kroB200_time[0]))
print("Min:", np.min(results_kroB200_time[0]))

In [ ]:
print("ILS1 kroA200 time:")
print("Mean:", np.mean(results_kroA200_time[1]))
print("Max:", np.max(results_kroA200_time[1]))
print("Min:", np.min(results_kroA200_time[1]))
print("ILS1 kroB200 time:")
print("Mean:", np.mean(results_kroB200_time[1]))
print("Max:", np.max(results_kroB200_time[1]))
print("Min:", np.min(results_kroB200_time[1]))

In [ ]:
print("ILS2 kroA200 time:")
print("Mean:", np.mean(results_kroA200_time[2]))
print("Max:", np.max(results_kroA200_time[2]))
print("Min:", np.min(results_kroA200_time[2]))
print("ILS2 kroB200 time:")
print("Mean:", np.mean(results_kroB200_time[2]))
print("Max:", np.max(results_kroB200_time[2]))
print("Min:", np.min(results_kroB200_time[2]))

In [ ]:
print("ILS2a kroA200 time:")
print("Mean:", np.mean(results_kroA200_time[3]))
print("Max:", np.max(results_kroA200_time[3]))
print("Min:", np.min(results_kroA200_time[3]))
print("ILS2a kroB200 time:")
print("Mean:", np.mean(results_kroB200_time[3]))
print("Max:", np.max(results_kroB200_time[3]))
print("Min:", np.min(results_kroB200_time[3]))

In [ ]:
print("MSLS kroA200 score:")
mean = []
for x in results_kroA200[0]:
    mean.append(x[0])
print("Mean:", np.mean(mean))
print("Max:", np.max(results_kroA200[0], axis=0)[0])
print("Min:", np.min(results_kroA200[0], axis=0)[0])
print("MSLS kroB200 score:")
mean = []
for x in results_kroB200[0]:
    mean.append(x[0])
print("Mean:", np.mean(mean))
print("Max:", np.max(results_kroB200[0], axis=0)[0])
print("Min:", np.min(results_kroB200[0], axis=0)[0])

In [ ]:
print("ILS1 kroA200 score:")
mean = []
for x in results_kroA200[1]:
    mean.append(x[0])
print("Mean:", np.mean(mean))
print("Max:", np.max(results_kroA200[1], axis=0)[0])
print("Min:", np.min(results_kroA200[1], axis=0)[0])
print("ILS1 kroB200 score:")
mean = []
for x in results_kroB200[1]:
    mean.append(x[0])
print("Mean:", np.mean(mean))
print("Max:", np.max(results_kroB200[1], axis=0)[0])
print("Min:", np.min(results_kroB200[1], axis=0)[0])

In [ ]:
print("ILS2 kroA200 score:")
mean = []
for x in results_kroA200[2]:
    mean.append(x[0])
print("Mean:", np.mean(mean))
print("Max:", np.max(results_kroA200[2], axis=0)[0])
print("Min:", np.min(results_kroA200[2], axis=0)[0])
print("ILS2 kroB200 score:")
mean = []
for x in results_kroB200[2]:
    mean.append(x[0])
print("Mean:", np.mean(mean))
print("Max:", np.max(results_kroB200[2], axis=0)[0])
print("Min:", np.min(results_kroB200[2], axis=0)[0])

In [ ]:
print("ILS2a kroA200 score:")
mean = []
for x in results_kroA200[3]:
    mean.append(x[0])
print("Mean:", np.mean(mean))
print("Max:", np.max(results_kroA200[3], axis=0)[0])
print("Min:", np.min(results_kroA200[3], axis=0)[0])
print("ILS2a kroB200 score:")
mean = []
for x in results_kroB200[3]:
    mean.append(x[0])
print("Mean:", np.mean(mean))
print("Max:", np.max(results_kroB200[3], axis=0)[0])
print("Min:", np.min(results_kroB200[3], axis=0)[0])

In [ ]:
plt.rc('figure', dpi=110, figsize=(9, 5))
sns.set_style('darkgrid')
    
for table in [results_kroA200[0],results_kroA200[1],results_kroA200[2],results_kroA200[3]]:    
    best_result = np.argmin(table, axis=0)
    cycle1 = table[best_result[0]][1]
    cycle2 = table[best_result[0]][2]

    cycle1.append(cycle1[0])
    cycle2.append(cycle2[0])

    coordinate_x_cycle1 = []
    coordinate_y_cycle1 = []
    coordinate_x_cycle2 = []
    coordinate_y_cycle2 = []
    for node in cycle1:
        coordinate_x_cycle1.append(instance.loc[node]['x'])
        coordinate_y_cycle1.append(instance.loc[node]['y'])
    for node in cycle2:
        coordinate_x_cycle2.append(instance.loc[node]['x'])
        coordinate_y_cycle2.append(instance.loc[node]['y'])

    # plotting the line 1 points
    plt.plot(coordinate_x_cycle1, coordinate_y_cycle1, '-bo',  c='blue', mfc='k', mec='k', label="cycle1")
    plt.plot(coordinate_x_cycle2, coordinate_y_cycle2, '-bo',  c='red', mfc='k', mec='k', label="cycle2")
    # naming the x axis
    plt.xlabel('x - axis')
    # naming the y axis
    plt.ylabel('y - axis')
    # giving a title to my graph

    # show a legend on the plot
    plt.legend()

    # function to show the plot
    plt.show()

In [ ]:
plt.rc('figure', dpi=110, figsize=(9, 5))
sns.set_style('darkgrid')
    
for table in [results_kroB200[0],results_kroB200[1],results_kroB200[2],results_kroB200[3]]:    
    best_result = np.argmin(table, axis=0)
    cycle1 = table[best_result[0]][1]
    cycle2 = table[best_result[0]][2]

    cycle1.append(cycle1[0])
    cycle2.append(cycle2[0])

    coordinate_x_cycle1 = []
    coordinate_y_cycle1 = []
    coordinate_x_cycle2 = []
    coordinate_y_cycle2 = []
    for node in cycle1:
        coordinate_x_cycle1.append(instance2.loc[node]['x'])
        coordinate_y_cycle1.append(instance2.loc[node]['y'])
    for node in cycle2:
        coordinate_x_cycle2.append(instance2.loc[node]['x'])
        coordinate_y_cycle2.append(instance2.loc[node]['y'])

    # plotting the line 1 points
    plt.plot(coordinate_x_cycle1, coordinate_y_cycle1, '-bo',  c='blue', mfc='k', mec='k', label="cycle1")
    plt.plot(coordinate_x_cycle2, coordinate_y_cycle2, '-bo',  c='red', mfc='k', mec='k', label="cycle2")
    # naming the x axis
    plt.xlabel('x - axis')
    # naming the y axis
    plt.ylabel('y - axis')
    # giving a title to my graph

    # show a legend on the plot
    plt.legend()

    # function to show the plot
    plt.show()